In [1]:
%run run_model.py --process=0 --rawname=1 --resume=0 --anchoring=0

24-02-05 06:28:15|MOD:run_model   |: Start Process [Load Data]!


--Process Queue : Data + Train + Test + Instance
--Start Training New!
--Model_name is set to LSTM_day_SHORTTEST!


24-02-05 06:28:30|MOD:run_model   |: Finish Process [Load Data]! Cost 14.6Secs
24-02-05 06:28:30|MOD:run_model   |: Start Process [Train Model]!


Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]


LSTM_day_SHORTTEST #0 @20170103 LoadData Cost    1.9Secs
FirstBite Ep#  0 : loss  1.00561, train-0.00553, valid-0.03184, max-0.0318, best-0.0318, lr1.0e-07


...

FirstBite Ep# 10 : loss  0.84852, train 0.16435, valid 0.15665, max 0.1566, best 0.1566, lr1.3e-03


None {'early_stop': False, 'max_epoch': False, 'valid_converge': False}
None {'early_stop': False, 'max_epoch': False, 'valid_converge': False}
None {'early_stop': False, 'max_epoch': False, 'valid_converge': False}
None {'early_stop': False, 'max_epoch': False, 'valid_converge': False}
None {'early_stop': False, 'max_epoch': False, 'valid_converge': False}


FirstBite Ep# 15 : loss  0.82406, train 0.19369, valid 0.17420, max 0.1742, best 0.1742, lr6.3e-04


None {'early_stop': False, 'max_epoch': False, 'valid_converge': False}
None {'early_stop': False, 'max_epoch': False, 'valid_converge': False}
None {'early_stop': False, 'max_epoch': False, 'valid_converge': False}
None {'early_stop': False, 'max_epoch': False, 'valid_converge': False}
Max Epoch {'early_stop': False, 'max_epoch': True, 'valid_converge': False}


24-02-05 06:28:58|MOD:run_model   |: LSTM_day_SHORTTEST #0 @20170704|Round 0 FirstBite Ep# 19 Max Epoch|Train 0.1984 Valid 0.1780 BestVal 0.1790|Cost  0.2Min,  0.6Sec/Ep
24-02-05 06:28:58|MOD:run_model   |: Finish Process [Train Model]! Cost 0.0 Hours, 0.2 Min/model, 0.7 Sec/Epoch
24-02-05 06:28:58|MOD:run_model   |: Start Process [Test Model]!
24-02-05 06:28:58|MOD:run_model   |: Each Model Date Testing Mean Score({'train': 'pearson', 'valid': 'pearson', 'test': 'pearson'}):
Models            0       0       0
Output         best swalast swabest
20170103     0.1045  0.1025  0.1063
20170704     0.1125  0.1003  0.1133
AllTimeAvg   0.1082  0.1015  0.1095
AllTimeSum    24.13   22.64   24.42
Std          0.0806  0.0720  0.0787
TValue        20.06   21.06   20.78
AnnIR        6.5800  6.9075  6.8174
24-02-05 06:29:09|MOD:run_model   |: Finish Process [Test Model]! Cost 11.0 Secs
24-02-05 06:29:09|MOD:run_model   |: Will not copy to instance!
